In [2]:
# imports
import pandas as p                              # it's pandas
from sklearn.preprocessing import MinMaxScaler  # for features scaling
import traceback                                # for error handling


In [4]:
# Import dataframe from "TREAD" notebook
%store -r
df = concatDF
routes_list = list_of_routes

test_route = routes_list[1]

In [6]:
# some info about the dataframe

display(test_route)
#display(df)
# current columns (aka features) of the dataset
print(df.columns)
display(df.info())

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,VesselType,Status,...,Draft,IsClassA,Avg_Speed,EstimatedStatus,IsInWP,IsInSignificantWP,Route,IsInRoute,group,route_id
556,338143427,2019-01-08 00:32:22,20.7358,-156.8890,6.9,116.000000,511.0,KOA,36,99,...,NaN,False,0.53,sailing,WP_0,<NA>,WP_0 - WP_33,True,1,2189
557,338143427,2019-01-08 00:33:52,20.7364,-156.8862,6.3,74.800003,511.0,KOA,36,99,...,NaN,False,6.46,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2189
558,338143427,2019-01-08 00:36:21,20.7376,-156.8818,6.0,76.300003,511.0,KOA,36,99,...,NaN,False,6.22,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2189
559,338143427,2019-01-08 00:37:22,20.7381,-156.8800,6.1,74.900002,511.0,KOA,36,99,...,NaN,False,6.23,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2189
560,338143427,2019-01-08 00:38:54,20.7388,-156.8774,6.1,64.500000,511.0,KOA,36,99,...,NaN,False,5.95,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,338143427,2019-01-08 03:13:51,20.8544,-156.6858,5.8,28.500000,511.0,KOA,36,99,...,NaN,False,6.01,sailing,WP_33,<NA>,WP_0 - WP_33,True,0,2189
657,338143427,2019-01-08 03:14:53,20.8558,-156.6850,6.0,27.200001,511.0,KOA,36,99,...,NaN,False,5.54,sailing,WP_33,<NA>,WP_0 - WP_33,True,0,2189
658,338143427,2019-01-08 03:16:22,20.8581,-156.6843,5.8,14.400000,511.0,KOA,36,99,...,NaN,False,5.81,sailing,WP_33,<NA>,WP_0 - WP_33,True,0,2189
659,338143427,2019-01-08 03:17:52,20.8606,-156.6840,6.2,13.300000,511.0,KOA,36,99,...,NaN,False,6.05,sailing,WP_33,<NA>,WP_0 - WP_33,True,0,2189


Index(['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',
       'VesselName', 'VesselType', 'Status', 'Length', 'Width', 'Draft',
       'IsClassA', 'Avg_Speed', 'EstimatedStatus', 'IsInWP',
       'IsInSignificantWP', 'Route', 'IsInRoute', 'group', 'route_id'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 5415129 entries, 0 to 2371
Data columns (total 22 columns):
 #   Column             Dtype        
---  ------             -----        
 0   MMSI               int32        
 1   BaseDateTime       datetime64[s]
 2   LAT                float64      
 3   LON                float64      
 4   SOG                float64      
 5   COG                float32      
 6   Heading            float32      
 7   VesselName         string       
 8   VesselType         int16        
 9   Status             int8         
 10  Length             int16        
 11  Width              int16        
 12  Draft              float32      
 13  IsClassA           boo

None

#### General Approach
1. Filling missing data? (To be decided)
2. Normalization (some features have different scales)
3. Dimensionality reduction (if possible)
4. Time series alignment? (To be decided)
5. Clustering algorithm

### Normalization

In [11]:
test_route.isnull().sum()

MMSI                  0
BaseDateTime          0
LAT                   0
LON                   0
SOG                   0
COG                   0
Heading               0
VesselName            0
VesselType            0
Status                0
Length                0
Width                 0
Draft                83
IsClassA              0
Avg_Speed             0
EstimatedStatus       0
IsInWP               16
IsInSignificantWP    83
Route                 0
IsInRoute             0
group                 0
group_id              0
route_id              0
dtype: int64

In [15]:
df = df.sort_values(by=['MMSI','BaseDateTime'])

scaler = MinMaxScaler()
cols_to_be_scaled = ['LAT','LON','SOG','COG','Heading','Length','Width','Avg_Speed']
df[cols_to_be_scaled] = scaler.fit_transform(df[cols_to_be_scaled])

display(df.head(3))

routes = df.groupby('MMSI').apply(lambda x : x[cols_to_be_scaled].values)

display(routes)

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,VesselType,Status,...,IsClassA,Avg_Speed,EstimatedStatus,IsInWP,IsInSignificantWP,Route,IsInRoute,group,group_id,route_id
37,338143427,2019-01-02 19:53:11,0.151496,0.262390,0.286307,0.319989,1.0,KOA,36,99,...,False,0.224818,sailing,WP_0,<NA>,WP_0 - WP_33,True,1,2181,2181
38,338143427,2019-01-02 19:58:08,0.150859,0.290666,0.261411,0.238522,1.0,KOA,36,99,...,False,0.223286,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2181,2181
39,338143427,2019-01-02 19:59:10,0.154042,0.296139,0.278008,0.174132,1.0,KOA,36,99,...,False,0.215243,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2181,2181


/tmp/ipykernel_11613/4013747349.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  routes = df.groupby('MMSI').apply(lambda x : x[cols_to_be_scaled].values)


MMSI
338143427    [[0.15149586250797142, 0.26238978412891356, 0....
366339000    [[0.14704010184595973, 0.2569169960474369, 0.0...
367745320    [[0.17759388924250175, 0.24688355123134897, 0....
dtype: object

### Dimensionality Reduction
In order to do reduce the number of features of the data source using the PCA methodology, some columns need to be converted into numerical form.
- 'BaseDateTime' from datetime64 to int64
- 'VesselName' -> column removed
- 'EstimatedStatus' from string to int (category)
- 'IsInWP' from string to int (category)
- 'IsInSignificantWP' from string to int (category)
- 'Route' -> Column manually removed (because it contains the same value)

In [3]:
df = df.drop(['VesselName', 'Route'], axis=1)

In [4]:
try:
    df['BaseDateTime'] = p.to_numeric(df['BaseDateTime'], errors='raise')
    df['EstimatedStatus'] = df['EstimatedStatus'].astype('category').cat.codes
    df['IsInWP'] = df['IsInWP'].astype('category').cat.codes
    df['IsInSignificantWP'] = df['IsInSignificantWP'].astype('category').cat.codes
    print(df.info())
except Exception:
    print(traceback.format_exc())

# search for any column with at least one NaN value
cols_with_nans = list(df.columns[df.isnull().any()])
if cols_with_nans:
    print(f"The columns {cols_with_nans} need their NaNs values fixed!)")
else:
    print('No NaNs values found')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10712 entries, 0 to 10711
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MMSI               10712 non-null  int32  
 1   BaseDateTime       10712 non-null  int64  
 2   LAT                10712 non-null  float64
 3   LON                10712 non-null  float64
 4   SOG                10712 non-null  float64
 5   COG                10712 non-null  float32
 6   Heading            10712 non-null  float32
 7   VesselType         10712 non-null  int16  
 8   Status             10712 non-null  int8   
 9   Length             10712 non-null  int16  
 10  Width              10712 non-null  int16  
 11  Draft              0 non-null      float32
 12  IsClassA           10712 non-null  bool   
 13  Avg_Speed          10712 non-null  float64
 14  EstimatedStatus    10712 non-null  int8   
 15  IsInWP             10712 non-null  int8   
 16  IsInSignificantWP  107

In [5]:
# Replacing NaN values
df['Draft'] = df['Draft'].fillna(value=0)

# search for any column with at least one NaN value
cols_with_nans = list(df.columns[df.isnull().any()])
if cols_with_nans:
    print(f"The columns {cols_with_nans} need their NaNs values fixed!)")
else:
    print('No NaNs values found')

No NaNs values found


In [6]:
'''# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
# pca dimensionality reduction
pca = decomposition.PCA(n_components='mle') # automatically detect the number of relevant features
#pca = decomposition.PCA(n_components=15)
reduced_data = pca.fit_transform(df)
reduced_df = p.DataFrame(reduced_data, columns=["Col_" + str(x) for x in range(0,15)])
display(reduced_df)
print(reduced_df.info())'''

'# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html\n# pca dimensionality reduction\npca = decomposition.PCA(n_components=\'mle\') # automatically detect the number of relevant features\n#pca = decomposition.PCA(n_components=15)\nreduced_data = pca.fit_transform(df)\nreduced_df = p.DataFrame(reduced_data, columns=["Col_" + str(x) for x in range(0,15)])\ndisplay(reduced_df)\nprint(reduced_df.info())'